# Discovery

By now, world population is in constant increase. The more the people, the more the food we need that translate in an increase of use of natural resources.


But which are the product we produce that exploit most resources? Which countries contribute to the exploitation of natural resources?

**Goal**: Discovering which are the top 10 products that use more water, need more land and emit more gas. Then discovering which countries are the most producer of each category


# Data Selection

In [479]:
import pandas as pd
import numpy as np
import sidetable 
from dataprep.clean import clean_country
from dataprep.clean import clean_df
from dataprep.clean import validate_lat_long
from dataprep.clean import clean_headers
import country_converter as coco
import plotly 
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go 


In [480]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.options.display.max_colwidth = 100
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:20,.2f}'.format)

In [481]:
fao_df = pd.read_csv('FAO.csv', encoding='latin-1')
food_pr_df = pd.read_csv('Food_Production.csv')

# Data Cleaning

### FAO Data

In [482]:
fao_df.head()

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,"1,928.00","1,904.00","1,666.00","1,950.00","2,001.00","1,808.00","2,053.00","2,045.00","2,154.00","1,819.00","1,963.00","2,215.00","2,310.00","2,335.00","2,434.00","2,512.00","2,282.00","2,454.00","2,443.00","2,129.00","2,133.00","2,068.00","1,994.00","1,851.00","1,791.00","1,683.00","2,194.00","1,801.00","1,754.00","1,640.00","1,539.00","1,582.00","1,840.00","1,855.00","1,853.00","2,177.00","2,343.00","2,407.00","2,463.00","2,600.00","2,668.00","2,776.00","3,095.00","3,249.00","3,486.00","3,704.00","4,164.00","4,252.00","4,538.00","4,605.00","4,711.00",4810,4895
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,183.00,183.00,182.00,220.00,220.00,195.00,231.00,235.00,238.00,213.00,205.00,233.00,246.00,246.00,255.00,263.00,235.00,254.00,270.00,259.00,248.00,217.00,217.00,197.00,186.00,200.00,193.00,202.00,191.00,199.00,197.00,249.00,218.00,260.00,319.00,254.00,326.00,347.00,270.00,372.00,411.00,448.00,460.00,419.00,445.00,546.00,455.00,490.00,415.00,442.00,476.00,425,422
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,76.00,76.00,76.00,76.00,76.00,75.00,71.00,72.00,73.00,74.00,71.00,70.00,72.00,76.00,77.00,80.00,60.00,65.00,64.00,64.00,60.00,55.00,53.00,51.00,48.00,46.00,46.00,47.00,46.00,43.00,43.00,40.00,50.00,46.00,41.00,44.00,50.00,48.00,43.00,26.00,29.00,70.00,48.00,58.00,236.00,262.00,263.00,230.00,379.00,315.00,203.00,367,360
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,237.00,237.00,237.00,238.00,238.00,237.00,225.00,227.00,230.00,234.00,223.00,219.00,225.00,240.00,244.00,255.00,185.00,203.00,198.00,202.00,189.00,174.00,167.00,160.00,151.00,145.00,145.00,148.00,145.00,135.00,132.00,120.00,155.00,143.00,125.00,138.00,159.00,154.00,141.00,84.00,83.00,122.00,144.00,185.00,43.00,44.00,48.00,62.00,55.00,60.00,72.00,78,89
4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,210.00,210.00,214.00,216.00,216.00,216.00,235.00,232.00,236.00,200.00,201.00,216.00,228.00,231.00,234.00,240.00,228.00,234.00,228.00,226.00,210.00,199.00,192.00,182.00,173.00,170.00,154.00,148.00,137.00,144.00,126.00,90.00,141.00,150.00,159.00,108.00,90.00,99.00,72.00,35.00,48.00,89.00,63.00,120.00,208.00,233.00,249.00,247.00,195.00,178.00,191.00,200,200


In [483]:
fao_df.shape

(21477, 63)

#### Checking columns dtypes

In [484]:
fao_dtypes, clean_fao = clean_df(fao_df, clean_header=False, standardize_missing_values='ignore', downcast_memory=False)

Data Type Detection Report:
	These data types are supported by DataPrep to clean: ['country', 'coordinate', 'address']


In [485]:
fao_dtypes

,semantic_data_type,atomic_data_type
Area Abbreviation,country,string
Area Code,integer,integer
Area,country,string
Item Code,integer,integer
Item,string,string
Element Code,integer,integer
Element,string,string
Unit,address,string
latitude,coordinate,floating
longitude,coordinate,floating


changing *Y2012* and *Y2013* data type from int to float

In [486]:
clean_fao[['Y2012', 'Y2013']] = clean_fao[['Y2012', 'Y2013']].astype(float)

#### Checking column names and eventually fixing them

In [487]:
clean_fao.columns

Index(['Area Abbreviation', 'Area Code', 'Area', 'Item Code', 'Item',
       'Element Code', 'Element', 'Unit', 'latitude', 'longitude', 'Y1961',
       'Y1962', 'Y1963', 'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969',
       'Y1970', 'Y1971', 'Y1972', 'Y1973', 'Y1974', 'Y1975', 'Y1976', 'Y1977',
       'Y1978', 'Y1979', 'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985',
       'Y1986', 'Y1987', 'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993',
       'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001',
       'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009',
       'Y2010', 'Y2011', 'Y2012', 'Y2013'],
      dtype='object')

In [488]:
#renaming columns in snake case and the years columns without the 'Y'
clean_fao = clean_headers(clean_fao, case='snake', replace={'Y':''})
clean_fao.columns

Column Headers Cleaning Report:
	61 values cleaned (96.83%)


Index(['area_abbreviation', 'area_code', 'area', 'item_code', 'item',
       'element_code', 'element', 'unit', 'latitude', 'longitude', '1961',
       '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970',
       '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979',
       '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988',
       '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013'],
      dtype='object')

#### Checking for duplicates

In [489]:
print(f'Number of rows before dropping duplicates: {clean_fao.shape[0] :>6}')
clean_fao.drop_duplicates(keep='first')
print(f'Number of rows after dropping duplicates: {clean_fao.shape[0] :>7}')

Number of rows before dropping duplicates:  21477
Number of rows after dropping duplicates:   21477


Counting the number of rows before and after checking for duplicates. No duplicates were found

#### Checking qualitative columns values

In [490]:
years = clean_fao.columns[clean_fao.columns.get_loc('1961'):]
clean_fao.stb.counts(exclude=['number'])

,count,unique,most_freq,most_freq_count,least_freq,least_freq_count
unit,21477,1,1000 tonnes,21477,1000 tonnes,21477
element,21477,2,Food,17528,Feed,3949
item,21477,115,Milk - Excluding Butter,558,"Meat, Aquatic Mammals",3
area_abbreviation,21477,169,CHN,541,LSO,75
area,21477,174,Spain,150,Lesotho,75


In [491]:
to_check = ['area', 'area_abbreviation', 'item']

for col in to_check:
    print(f'{col}\n{sorted(clean_fao[col].unique())}', end=f'\n{"-"*200}\n')

area
['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China, Hong Kong SAR', 'China, Macao SAR', 'China, Taiwan Province of', 'China, mainland', 'Colombia', 'Congo', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czechia', "Côte d'Ivoire", "Democratic People's Republic of Korea", 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Estonia', 'Ethiopia', 'Fiji', 'Finland', 'France', 'French Polynesia', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hungary', 'Icel

#### Fixing Area column values

After a close look, some of the Area names are not correct. let's correct them:

With the function clean_country *Taiwan*, *Macau* and *Hong Kong* would fall under *China*, let's remove it from those country before applying the function

In [492]:
clean_fao['area'] = clean_fao['area'].apply(lambda x: 'Taiwan' if 'Taiwan' in x else x)
clean_fao['area'] = clean_fao['area'].apply(lambda x: 'Hong Kong' if 'Hong Kong' in x else x)
clean_fao['area'] = clean_fao['area'].apply(lambda x: 'Macau' if 'Macao' in x else x)

Cleaning country names with clean_country

In [493]:
clean_fao = clean_country(clean_fao, 'area', input_format=('name', 'official'), output_format='official', fuzzy_dist=2, inplace=True, errors='raise')

  0%|          | 0/9 [00:00<?, ?it/s]

Country Cleaning Report:
	17246 values cleaned (80.3%)
Result contains 21477 (100.0%) values in the correct format and 0 null values (0.0%)


 *Area Abbreviation* and the *Area* count do not match. Let's investigate why

In [494]:
# checking if there are more than 1 country with the same area code
a_code_unique = clean_fao.groupby('area_abbreviation')['area_clean'].nunique()
a_code_unique[a_code_unique > 1]

area_abbreviation
AZE    2
CHN    4
THA    2
Name: area_clean, dtype: int64

As suspected, 3 *Area Abbreviation* are associated with more than 1 country. Let's find out which countries fall under the same code

In [495]:
cond = clean_fao['area_abbreviation'].isin(['AZE', 'THA', 'CHN'])
clean_fao.loc[cond].groupby('area_abbreviation')['area_clean'].unique()

area_abbreviation
AZE                        [Republic of Azerbaijan, Commonwealth of the Bahamas]
CHN    [Hong Kong SAR, Macau SAR, People's Republic of China, Republic of China]
THA                                 [Kingdom of Thailand, Republic of Macedonia]
Name: area_clean, dtype: object

The result of the investigation evidenciate that *Bahamas*, *Macedonia*, *Taiwan*, *Macau* and *Hong Kong* fall under the wrong *Area abbreviation* code.

Fixing the wrong Area Abbreviation values

In [496]:
cond = clean_fao['area_clean'] == 'Bahamas'
clean_fao.loc[cond, 'area_abbreviation'] = 'BHS'

cond = clean_fao['area_clean'] == 'Hong Kong'
clean_fao.loc[cond, 'area_abbreviation'] = 'HKG'

cond = clean_fao['area_clean'] == 'Macau'
clean_fao.loc[cond, 'area_abbreviation'] = 'MAC'

cond = clean_fao['area_clean'] == 'Taiwan'
clean_fao.loc[cond, 'area_abbreviation'] = 'TWN'

cond = clean_fao['area_clean'] == 'Macedonia'
clean_fao.loc[cond, 'area_abbreviation'] = 'MKD'

#### Dropping unecessary columns

In [497]:
clean_fao.drop(columns=['area_code', 'item_code', 'element_code', 'unit'], inplace=True)

#### Checking latitude and longitude values

Let's check if latitude and longitude contains some non valide values. 

In [498]:
validate_lat_long(clean_fao["latitude"], lat_long=False, lat=True).all()

True

In [499]:
validate_lat_long(clean_fao["longitude"], lat_long=False, lon=True).all()

True

Latitude ranges from -90 to 90,  longitude ranges from -180 to 180

In [500]:
clean_fao[['longitude', 'latitude']].agg([min, max])

,longitude,latitude
min,-172.10,-40.90
max,179.41,64.96


Both max and min of latitude and longitude fall under the accepted range

#### Checking if there are some invalid values in the *Years* columns

Let's check if are there any negative numbers in production

In [501]:
cond = clean_fao.loc[:,'1961':'2013'].agg([min])
cond.T[cond.T['min'] < 0]

,min
2012,-169.00
2013,-246.00


Now let's investigate which country has a negative amount of production for which *Item* and *Element* (food or feed)

In [502]:
clean_fao.loc[clean_fao[['2013', '2012']].idxmin().unique(), ['area_clean', 'item', 'element']]

,area_clean,item,element
10082,Japan,Oats,Food


Negative number in production must be an error of input, just drop the entire row

In [503]:
clean_fao.drop(labels=10082, inplace=True)

#### Checking for missing values

In [504]:
clean_fao.stb.missing(clip_0=True, style=True)

,missing,total,percent
1961,"3,539","21,476",16.48%
1976,"3,539","21,476",16.48%
1989,"3,539","21,476",16.48%
1988,"3,539","21,476",16.48%
1987,"3,539","21,476",16.48%
1962,"3,539","21,476",16.48%
1985,"3,539","21,476",16.48%
1984,"3,539","21,476",16.48%
1983,"3,539","21,476",16.48%
1982,"3,539","21,476",16.48%


Let's create a dataframe containing only the years variables

In [505]:
year_df = clean_fao[years]

First let's drop all the rows that contain all NaN values, if there are any

In [506]:
clean_fao = clean_fao.dropna(how='all')

Since the only NaN values in the dataframe are in the numeric columns, let's fill all the remaining NaN values with 0

In [507]:
clean_fao.fillna(0, inplace=True)

Lastly we  create a list of index of the rows of the years dataframe that has only 0 values

In [508]:
idx_to_drop = year_df.loc[(year_df == 0).all(axis=1)].index

Finally droping those rows

In [509]:
clean_fao = clean_fao.drop(labels=idx_to_drop)

Updating the years dataset with the NaN value dropped

In [510]:
year_df = clean_fao[years]

### Food Production Data

In [511]:
food_pr_df.head()

,Food product,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail,Total_emissions,Eutrophying emissions per 1000kcal (gPO₄eq per 1000kcal),Eutrophying emissions per kilogram (gPO₄eq per kilogram),Eutrophying emissions per 100g protein (gPO₄eq per 100 grams protein),Freshwater withdrawals per 1000kcal (liters per 1000kcal),Freshwater withdrawals per 100g protein (liters per 100g protein),Freshwater withdrawals per kilogram (liters per kilogram),Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal),Greenhouse gas emissions per 100g protein (kgCO₂eq per 100g protein),Land use per 1000kcal (m² per 1000kcal),Land use per kilogram (m² per kilogram),Land use per 100g protein (m² per 100g protein),Scarcity-weighted water use per kilogram (liters per kilogram),Scarcity-weighted water use per 100g protein (liters per 100g protein),Scarcity-weighted water use per 1000kcal (liters per 1000 kilocalories)
0,Wheat & Rye (Bread),0.10,0.00,0.80,0.20,0.10,0.10,0.10,1.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Maize (Meal),0.30,0.00,0.50,0.10,0.10,0.10,0.00,1.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Barley (Beer),0.00,0.00,0.20,0.10,0.00,0.50,0.30,1.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Oatmeal,0.00,0.00,1.40,0.00,0.10,0.10,0.00,1.60,4.28,11.23,8.64,183.91,371.08,482.40,0.95,1.91,2.90,7.60,5.85,"18,786.20","14,450.92","7,162.10"
4,Rice,0.00,0.00,3.60,0.10,0.10,0.10,0.10,4.00,9.51,35.07,49.39,609.98,"3,166.76","2,248.40",1.21,6.27,0.76,2.80,3.94,"49,576.30","69,825.77","13,449.89"


In [512]:
food_pr_df.describe()

,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail,Total_emissions,Eutrophying emissions per 1000kcal (gPO₄eq per 1000kcal),Eutrophying emissions per kilogram (gPO₄eq per kilogram),Eutrophying emissions per 100g protein (gPO₄eq per 100 grams protein),Freshwater withdrawals per 1000kcal (liters per 1000kcal),Freshwater withdrawals per 100g protein (liters per 100g protein),Freshwater withdrawals per kilogram (liters per kilogram),Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal),Greenhouse gas emissions per 100g protein (kgCO₂eq per 100g protein),Land use per 1000kcal (m² per 1000kcal),Land use per kilogram (m² per kilogram),Land use per 100g protein (m² per 100g protein),Scarcity-weighted water use per kilogram (liters per kilogram),Scarcity-weighted water use per 100g protein (liters per 100g protein),Scarcity-weighted water use per 1000kcal (liters per 1000 kilocalories)
count,43.00,43.00,43.00,43.00,43.00,43.00,43.00,43.00,33.00,38.00,27.00,30.00,26.00,38.00,33.00,27.00,33.00,38.00,27.00,38.00,26.00,30.00
mean,1.26,0.45,3.47,0.25,0.20,0.27,0.07,5.97,27.18,46.14,52.77,504.19,"1,437.98",932.61,5.63,13.52,12.42,29.26,29.11,"36,607.43","59,196.44","17,380.58"
std,3.36,0.92,7.08,0.37,0.16,0.34,0.11,10.50,46.45,82.81,52.03,539.13,"1,441.98","1,297.00",10.61,19.43,28.35,78.49,49.31,"56,891.28","89,928.19","16,232.08"
min,-2.10,0.00,0.10,0.00,0.00,0.00,0.00,0.20,0.71,0.69,3.38,0.72,32.38,0.00,0.07,0.26,0.27,0.33,3.00,0.00,421.25,4.10
25%,0.00,0.00,0.35,0.00,0.10,0.10,0.00,0.85,4.21,3.75,17.86,106.93,373.57,105.50,0.63,4.03,1.31,1.11,5.09,"3,325.07","11,018.40","2,969.12"
50%,0.20,0.00,0.80,0.10,0.10,0.10,0.00,1.60,7.00,11.46,37.33,338.06,"1,083.33",417.10,1.35,6.50,2.98,6.87,7.94,"14,533.05","20,917.21","12,605.26"
75%,0.80,0.00,2.20,0.30,0.20,0.30,0.15,6.00,26.32,45.84,55.30,694.81,"1,832.39","1,340.38",5.34,14.98,6.61,14.92,23.00,"35,960.18","70,651.72","28,056.47"
max,16.30,2.90,39.40,1.30,0.80,1.60,0.30,59.60,197.36,365.29,185.05,"2,062.18","6,003.33","5,605.20",50.95,93.30,119.49,369.81,184.81,"229,889.80","431,620.00","49,735.88"


In [513]:
food_pr_df.shape

(43, 23)

In [514]:
food_pr_df.columns

Index(['Food product', 'Land use change', 'Animal Feed', 'Farm', 'Processing',
       'Transport', 'Packging', 'Retail', 'Total_emissions',
       'Eutrophying emissions per 1000kcal (gPO₄eq per 1000kcal)',
       'Eutrophying emissions per kilogram (gPO₄eq per kilogram)',
       'Eutrophying emissions per 100g protein (gPO₄eq per 100 grams protein)',
       'Freshwater withdrawals per 1000kcal (liters per 1000kcal)',
       'Freshwater withdrawals per 100g protein (liters per 100g protein)',
       'Freshwater withdrawals per kilogram (liters per kilogram)',
       'Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal)',
       'Greenhouse gas emissions per 100g protein (kgCO₂eq per 100g protein)',
       'Land use per 1000kcal (m² per 1000kcal)',
       'Land use per kilogram (m² per kilogram)',
       'Land use per 100g protein (m² per 100g protein)',
       'Scarcity-weighted water use per kilogram (liters per kilogram)',
       'Scarcity-weighted water use per 100g protein 

In [515]:
food_pr_df = clean_headers(food_pr_df, case='snake')
food_pr_df.columns

Column Headers Cleaning Report:
	23 values cleaned (100.0%)


Index(['food_product', 'land_use_change', 'animal_feed', 'farm', 'processing',
       'transport', 'packging', 'retail', 'total_emissions',
       'eutrophying_emissions_per_1000kcal_g_p_oeq_per_1000kcal',
       'eutrophying_emissions_per_kilogram_g_p_oeq_per_kilogram',
       'eutrophying_emissions_per_100g_protein_g_p_oeq_per_100_grams_protein',
       'freshwater_withdrawals_per_1000kcal_liters_per_1000kcal',
       'freshwater_withdrawals_per_100g_protein_liters_per_100g_protein',
       'freshwater_withdrawals_per_kilogram_liters_per_kilogram',
       'greenhouse_gas_emissions_per_1000kcal_kg_c_oeq_per_1000kcal',
       'greenhouse_gas_emissions_per_100g_protein_kg_c_oeq_per_100g_protein',
       'land_use_per_1000kcal_m_per_1000kcal',
       'land_use_per_kilogram_m_per_kilogram',
       'land_use_per_100g_protein_m_per_100g_protein',
       'scarcity_weighted_water_use_per_kilogram_liters_per_kilogram',
       'scarcity_weighted_water_use_per_100g_protein_liters_per_100g_protei

Dropping unecessary columns

In [516]:
to_drop = [
       'eutrophying_emissions_per_1000kcal_g_p_oeq_per_1000kcal',
       'eutrophying_emissions_per_100g_protein_g_p_oeq_per_100_grams_protein',
       'freshwater_withdrawals_per_1000kcal_liters_per_1000kcal',
       'freshwater_withdrawals_per_100g_protein_liters_per_100g_protein',
       'greenhouse_gas_emissions_per_1000kcal_kg_c_oeq_per_1000kcal',
       'greenhouse_gas_emissions_per_100g_protein_kg_c_oeq_per_100g_protein',
       'land_use_per_1000kcal_m_per_1000kcal',
       'land_use_per_100g_protein_m_per_100g_protein',
       'scarcity_weighted_water_use_per_100g_protein_liters_per_100g_protein',
       'scarcity_weighted_water_use_per_1000kcal_liters_per_1000_kilocalories'
       ]
food_pr_df.drop(to_drop, axis=1, inplace=True)



Removing missing values

In [517]:
food_pr_df.dropna(how='any', inplace=True)

In [518]:
#checking the negative values for the land_use_change columns as seen in df.describe()
cond = food_pr_df['land_use_change'] < 0
food_pr_df.loc[cond, :]

,food_product,land_use_change,animal_feed,farm,processing,transport,packging,retail,total_emissions,eutrophying_emissions_per_kilogram_g_p_oeq_per_kilogram,freshwater_withdrawals_per_kilogram_liters_per_kilogram,land_use_per_kilogram_m_per_kilogram,scarcity_weighted_water_use_per_kilogram_liters_per_kilogram
11,Nuts,-2.10,0.00,2.10,0.00,0.10,0.10,0.00,0.20,19.15,"4,133.80",12.96,"229,889.80"
19,Olive Oil,-0.40,0.00,4.30,0.70,0.50,0.90,0.00,6.00,37.26,"2,141.80",26.31,"177,480.20"
25,Citrus Fruit,-0.10,0.00,0.30,0.00,0.10,0.00,0.00,0.30,2.24,82.70,0.86,"4,662.70"
29,Wine,-0.10,0.00,0.60,0.10,0.10,0.70,0.00,1.40,4.57,78.90,1.78,"1,149.30"


In [519]:
#removing those values
#food_pr_df.drop(labels=[19, 25, 29], inplace=True)

# Data Exploration/Data Transformation

# Eutrophy

In [520]:
fig = px.histogram(food_pr_df, x=food_pr_df['eutrophying_emissions_per_kilogram_g_p_oeq_per_kilogram'],
                   marginal="box",
                   hover_data=food_pr_df.columns)
fig.show()

75% of the product produce less than 50 gPO₄eq per kilogram

Since we want to know the top product that contributes to the Eutrophyng emissions per kilogram, we first find a threshold above which focussing the analysys.
The threshold is set above the 3rd percentile: 

In [521]:
cond = food_pr_df['eutrophying_emissions_per_kilogram_g_p_oeq_per_kilogram'] > np.percentile(food_pr_df['eutrophying_emissions_per_kilogram_g_p_oeq_per_kilogram'], 75)
top_eutrophy_df = food_pr_df.loc[cond, ['food_product', 'eutrophying_emissions_per_kilogram_g_p_oeq_per_kilogram']].sort_values(by='eutrophying_emissions_per_kilogram_g_p_oeq_per_kilogram', ascending=False)
top_eutrophy_df

,food_product,eutrophying_emissions_per_kilogram_g_p_oeq_per_kilogram
34,Beef (dairy herd),365.29
33,Beef (beef herd),301.41
41,Fish (farmed),235.12
31,Coffee,110.52
39,Cheese,98.37
35,Lamb & Mutton,97.13
32,Dark Chocolate,87.08
36,Pig Meat,76.38
17,Sunflower Oil,50.66
37,Poultry Meat,48.70


In [522]:
top_emissions = food_pr_df.sort_values(by='eutrophying_emissions_per_kilogram_g_p_oeq_per_kilogram', ascending=False)[:10]['food_product'].to_list()

In [523]:
fig = px.bar(top_eutrophy_df, x='eutrophying_emissions_per_kilogram_g_p_oeq_per_kilogram', y='food_product')
fig.update_layout(yaxis={'categoryorder':'total ascending'})

fig.show()

# Freshwater withdrawals

In [524]:
fig = px.histogram(food_pr_df, x=food_pr_df['freshwater_withdrawals_per_kilogram_liters_per_kilogram'],
                   marginal="box",
                   hover_data=food_pr_df.columns)
fig.show()

In [525]:
cond = food_pr_df['freshwater_withdrawals_per_kilogram_liters_per_kilogram'] > np.percentile(food_pr_df['freshwater_withdrawals_per_kilogram_liters_per_kilogram'], 75)
top_water_df = food_pr_df.loc[cond, ['food_product', 'freshwater_withdrawals_per_kilogram_liters_per_kilogram']].sort_values(by='freshwater_withdrawals_per_kilogram_liters_per_kilogram', ascending=False)
top_water_df

,food_product,freshwater_withdrawals_per_kilogram_liters_per_kilogram
39,Cheese,"5,605.20"
11,Nuts,"4,133.80"
41,Fish (farmed),"3,691.30"
34,Beef (dairy herd),"2,714.30"
4,Rice,"2,248.40"
19,Olive Oil,"2,141.80"
12,Groundnuts,"1,852.30"
35,Lamb & Mutton,"1,802.80"
36,Pig Meat,"1,795.80"
33,Beef (beef herd),"1,451.20"


In [526]:
fig = px.bar(top_water_df, x='freshwater_withdrawals_per_kilogram_liters_per_kilogram', y='food_product')
fig.update_layout(yaxis={'categoryorder':'total ascending'})

fig.show()

In [527]:
top_water = food_pr_df.sort_values(by='freshwater_withdrawals_per_kilogram_liters_per_kilogram', ascending=False)[:10]['food_product'].to_list()

Top 10 prodcuts for land use

In [528]:
fig = px.histogram(food_pr_df, x=food_pr_df['land_use_per_kilogram_m_per_kilogram'],
                   marginal="box",
                   hover_data=food_pr_df.columns)
fig.show()

In [529]:
cond = food_pr_df['land_use_per_kilogram_m_per_kilogram'] > np.percentile(food_pr_df['land_use_per_kilogram_m_per_kilogram'], 75)
top_land_df = food_pr_df.loc[cond, ['food_product', 'land_use_per_kilogram_m_per_kilogram']].sort_values(by='land_use_per_kilogram_m_per_kilogram', ascending=False)
top_land_df

,food_product,land_use_per_kilogram_m_per_kilogram
35,Lamb & Mutton,369.81
33,Beef (beef herd),326.21
39,Cheese,87.79
32,Dark Chocolate,68.96
34,Beef (dairy herd),43.24
19,Olive Oil,26.31
31,Coffee,21.62
17,Sunflower Oil,17.66
36,Pig Meat,17.36
9,Other Pulses,15.57


In [530]:
fig = px.bar(top_land_df, x='land_use_per_kilogram_m_per_kilogram', y='food_product')
fig.update_layout(yaxis={'categoryorder':'total ascending'})

fig.show()

In [531]:
top_land = food_pr_df.sort_values(by='land_use_per_kilogram_m_per_kilogram', ascending=False)[:10]['food_product'].to_list()

# Scarcity-weighted water use

In [532]:
fig = px.histogram(food_pr_df, x=food_pr_df['scarcity_weighted_water_use_per_kilogram_liters_per_kilogram'],
                   marginal="box",
                   hover_data=food_pr_df.columns)
fig.show()

In [533]:
cond = food_pr_df['scarcity_weighted_water_use_per_kilogram_liters_per_kilogram'] > np.percentile(food_pr_df['scarcity_weighted_water_use_per_kilogram_liters_per_kilogram'], 75)
top_scarc_df = food_pr_df.loc[cond, ['food_product', 'scarcity_weighted_water_use_per_kilogram_liters_per_kilogram']].sort_values(by='scarcity_weighted_water_use_per_kilogram_liters_per_kilogram', ascending=False)
top_scarc_df

,food_product,scarcity_weighted_water_use_per_kilogram_liters_per_kilogram
11,Nuts,"229,889.80"
39,Cheese,"180,850.60"
19,Olive Oil,"177,480.20"
35,Lamb & Mutton,"141,925.00"
34,Beef (dairy herd),"119,805.20"
36,Pig Meat,"66,867.40"
12,Groundnuts,"61,797.90"
4,Rice,"49,576.30"
41,Fish (farmed),"41,572.20"
17,Sunflower Oil,"36,369.40"


In [534]:
fig = px.bar(top_scarc_df, x='scarcity_weighted_water_use_per_kilogram_liters_per_kilogram', y='food_product')
fig.update_layout(yaxis={'categoryorder':'total ascending'})

fig.show()

In [535]:
top_scarc = food_pr_df.sort_values(by='scarcity_weighted_water_use_per_kilogram_liters_per_kilogram', ascending=False)[:10]['food_product'].to_list()

# Green House emissions

In [536]:
fig = px.histogram(food_pr_df, x=food_pr_df['total_emissions'],
                   marginal="violin",
                   hover_data=food_pr_df.columns)
fig.show()

In [537]:
cond = food_pr_df['total_emissions'] > np.percentile(food_pr_df['total_emissions'], 75)
top_emiss_df = food_pr_df.loc[cond, ['food_product', 'total_emissions']].sort_values(by=('total_emissions'), ascending=False)
top_emiss_df

,food_product,total_emissions
33,Beef (beef herd),59.60
35,Lamb & Mutton,24.50
39,Cheese,21.20
34,Beef (dairy herd),21.10
32,Dark Chocolate,18.70
31,Coffee,16.50
16,Palm Oil,7.60
36,Pig Meat,7.20
37,Poultry Meat,6.10


In [538]:
fig = px.bar(top_emiss_df, x='total_emissions',y='food_product', orientation='h')

fig.update_layout(yaxis={'categoryorder':'total ascending'})

fig.show()

In [539]:
top_emiss = food_pr_df.sort_values(by='total_emissions', ascending=False)[:10]['food_product'].to_list()

In [540]:
#grafico temporaneo stacked per contributo di emissioni in percentuale

cond=food_pr_df['food_product'].isin(top_emiss)
df = food_pr_df.loc[cond, 'food_product':'total_emissions'].sort_values(by='total_emissions', ascending=False)
df = df.drop(columns='total_emissions')
df = df.set_index('food_product')
df = df.div(df.sum(axis=1), axis=0)*100
df = df.reset_index()

colors = ["#b30000", "#7c1158", "#4421af", "#1a53ff", "#0d88e6", "#00b7c7", "#5ad45a", "#8be04e", "#ebdc78"]

fig = px.bar(
    df, 
    y=['land_use_change', 'animal_feed','farm' , 'processing', 'transport', 'packging', 'retail'], 
    x='food_product', 
    title="CO2 emissions",
    color_discrete_sequence=colors,
    text_auto= '.0f',
    barmode='group',
    orientation='v'  
    )

fig.update_traces(textposition='outside')
    
fig.update_layout(
    title = 'Top 10 food items for Total Emissions of CO₂ - Production Steps',     
    title_x = 0.5,     
    title_y = 0.9,     
    title_xanchor='center',     
    title_yanchor='top',
    yaxis = dict(title='Emissions (kgCO₂ per kg of product)'),
    xaxis = dict(title=None),
    )
    
fig.show()

# FAO

Creating a column with the relative continent for each state

In [541]:
converter = coco.CountryConverter()

# creating a list for unique values of countries ISO3 and a list for the continent associated to each country 
lst = clean_fao['area_abbreviation'].unique()
continent = converter.convert(names = lst, src='ISO3', to='continent')

#joining those list in a dictionary
dic = {key:value for key, value in zip(lst, continent)}

# creating a column in the dataframe 
clean_fao['continent'] = clean_fao['area_abbreviation'].apply(lambda x: dic.get(x))


Calculating the total production for item from 1961 to 2012

In [542]:
clean_fao['total_production'] = clean_fao[years].sum(axis=1)

Calculating the mean of production from the first year of production of the item

In [543]:
#np.trim_zeros with args='f' trims zeros before the first non null value
clean_fao['mean'] = year_df.apply(np.trim_zeros, axis=1, args='f').mean(axis=1)

In [544]:
#melting the dataframe from wide to long
long_fao = clean_fao.melt(
    id_vars=['area_abbreviation', 'area_clean', 'item', 'element', 'latitude', 'longitude', 'continent'],
    value_vars=[str(n) for n in range(1961, 2014)],
    var_name='year',
    value_name='production',
)

long_fao.head()

,area_abbreviation,area_clean,item,element,latitude,longitude,continent,year,production
0,AFG,Islamic Republic of Afghanistan,Wheat and products,Food,33.94,67.71,Asia,1961,"1,928.00"
1,AFG,Islamic Republic of Afghanistan,Rice (Milled Equivalent),Food,33.94,67.71,Asia,1961,183.00
2,AFG,Islamic Republic of Afghanistan,Barley and products,Feed,33.94,67.71,Asia,1961,76.00
3,AFG,Islamic Republic of Afghanistan,Barley and products,Food,33.94,67.71,Asia,1961,237.00
4,AFG,Islamic Republic of Afghanistan,Maize and products,Feed,33.94,67.71,Asia,1961,210.00


In [545]:
feed_food = long_fao.groupby(by=['element', 'year'])['production'].sum().reset_index()

fig = px.line(feed_food, x='year', y='production', color='element')
fig.show()

In [546]:
cond = feed_food['year'].isin(['1961','1992', '2013'])
pct_change50 = feed_food.loc[cond ,['element','year', 'production']]
pct_change50

,element,year,production
0,Feed,1961,"884,314.00"
31,Feed,1992,"1,986,016.00"
52,Feed,2013,"2,608,511.00"
53,Food,1961,"2,618,297.00"
84,Food,1992,"5,923,272.00"
105,Food,2013,"9,752,983.00"


In [547]:
# grafico temporaneo dell'incremnto percentuale feed and food
fig = px.bar(pct_change50, y='year', x='production', color='element', barmode='group', text='production')
fig.update_layout(yaxis={'categoryorder':'total descending'})
fig.show()

In [564]:
#trovare quali elementi sono stati i più prodotti per ogni anno e da quali paesi

# Data Visualization

In [ ]:
# dataset popolazione, dataset km2 di terra
# normalizzare produzione con popolazione
# normalizzare uso terra con km2
# trovare i top 10 prodotti che consumano piu risorse
# trovare i top 10 stati che producono quei prodotti